# Definição de bibliotecas e funções úteis

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DateType, FloatType, IntegerType, DoubleType
from datetime import datetime

# Prova Equipe de Dados

Somos a CantuStore: Plataforma de tecnologia e logística que viabiliza soluções completas em pneus, 
guiando quem compra e apoiando quem vende. Se o assunto é pneu, você resolve aqui. Produtos e 
serviços em uma experiência 360° para abrir caminhos e ver pessoas e negócios evoluindo junto com 
a gente. Afinal, ficar parado não é opção, pelos menos pra nós. 

## Parte 1 - SQL

---
# 🏆 Estrutura do Campeonato

A seguir, temos a modelagem e os dados de um campeonato entre equipes:

---

## 📄 Tabela `times`

```sql
CREATE TABLE times (
    time_id INTEGER NOT NULL,
    time_nome VARCHAR NOT NULL,
    UNIQUE(time_id)
);
````

### 📌 Dados da Tabela `times`

| Time\_id | Time\_nome |
| -------- | ---------- |
| 10       | Financeiro |
| 20       | Marketing  |
| 30       | Logística  |
| 40       | TI         |
| 50       | Dados      |

---

## 📄 Tabela `jogos`

```sql
CREATE TABLE jogos (
    jogo_id INTEGER NOT NULL,
    mandante_time INTEGER NOT NULL,
    visitante_time INTEGER NOT NULL,
    mandante_gols INTEGER NOT NULL,
    visitante_gols INTEGER NOT NULL,
    UNIQUE(jogo_id)
);
```

### 📌 Dados da Tabela `jogos`

| Jogo\_id | Time\_mandante | Time\_visitante | Gols\_mandante | Gols\_visitante |
| -------- | -------------- | --------------- | -------------- | --------------- |
| 1        | 30             | 20              | 1              | 0               |
| 2        | 10             | 20              | 1              | 2               |
| 3        | 20             | 50              | 2              | 2               |
| 4        | 10             | 30              | 1              | 1               |
| 5        | 30             | 50              | 0              | 1               |

---

## ⚙️ Regras para Pontuação

* ✅ Vitória: 3 pontos (mais gols que o adversário)
* ⚖️ Empate: 1 ponto (mesmo número de gols)
* ❌ Derrota: 0 pontos (menos gols que o adversário)

---

## 🎯 Objetivo

Escrever uma consulta que calcule o total de pontos de cada equipe (`time_id`) com base nas regras acima.

### Requisitos da consulta:

* Retornar o nome do time e o total de pontos (`num_pontos`);
* Agrupar por time;
* Ordenar por `num_pontos` em ordem decrescente;
* Em caso de empate, ordenar por `time_id` em ordem crescente.

---





#### 1.1.1 Cria tabelas no Databricks - com Unity Catalog

In [0]:
%sql
USE datacraft_catalog.cantu;
CREATE TABLE IF NOT EXISTS cantu.times (
  time_id LONG NOT NULL, 
  time_nome STRING NOT NULL,
  PRIMARY KEY(time_id)
);

In [0]:
%sql
USE datacraft_catalog.cantu;
CREATE TABLE IF NOT EXISTS cantu.jogos (
  jogo_id LONG NOT NULL, 
  mandante_time LONG NOT NULL,
  visitante_time LONG NOT NULL,
  mandante_gols LONG NOT NULL,
  visitante_gols LONG NOT NULL,
  PRIMARY KEY(jogo_id),
  CONSTRAINT fk_mandante_time FOREIGN KEY (mandante_time) REFERENCES times(time_id),
  CONSTRAINT fk_visitante_time FOREIGN KEY (visitante_time) REFERENCES times(time_id)
);

#### 1.1.2 Insere os dados nas tabelas no Databricks - com Unity Catalog

In [0]:
# 📄 DataFrame de times
df_times = spark.createDataFrame([
    (10, "Financeiro"),
    (20, "Marketing"),
    (30, "Logística"),
    (40, "TI"),
    (50, "Dados"),
], ["time_id", "time_nome"])

# 📄 DataFrame de jogos
df_jogos = spark.createDataFrame([
    (1, 30, 20, 1, 0),
    (2, 10, 20, 1, 2),
    (3, 20, 50, 2, 2),
    (4, 10, 30, 1, 0),
    (5, 30, 50, 0, 1),
], ["jogo_id", "mandante_time", "visitante_time", "mandante_gols", "visitante_gols"])

# Cria view temporária
df_times.createOrReplaceTempView('temp_times')
df_jogos.createOrReplaceTempView('temp_jogos')


In [0]:
%sql
INSERT INTO datacraft_catalog.cantu.times (
    time_id, time_nome
  )
SELECT
  time_id,
  time_nome
FROM temp_times

num_affected_rows,num_inserted_rows
5,5


In [0]:
# %sql
# TRUNCATE TABLE datacraft_catalog.cantu.times

In [0]:
%sql
INSERT INTO datacraft_catalog.cantu.jogos (
    jogo_id, mandante_time, visitante_time, mandante_gols, visitante_gols
  )
  SELECT
    jogo_id,
    mandante_time,
    visitante_time,
    mandante_gols,
    visitante_gols
  FROM
    temp_jogos

num_affected_rows,num_inserted_rows
5,5


In [0]:
# %sql
# TRUNCATE TABLE datacraft_catalog.cantu.jogos

In [0]:
%sql
WITH resultados_mandante AS (
    SELECT 
        mandante_time AS time_id,
        CASE 
            WHEN mandante_gols > visitante_gols THEN 3
            WHEN mandante_gols = visitante_gols THEN 1
            ELSE 0
        END AS pontos
    FROM datacraft_catalog.cantu.jogos
),
resultados_visitante AS (
    SELECT 
        visitante_time AS time_id,
        CASE 
            WHEN visitante_gols > mandante_gols THEN 3
            WHEN visitante_gols = mandante_gols THEN 1
            ELSE 0
        END AS pontos
    FROM datacraft_catalog.cantu.jogos
),
todos_resultados AS (
    SELECT * FROM resultados_mandante
    UNION ALL
    SELECT * FROM resultados_visitante
),
pontuacao_por_time AS (
    SELECT 
        time_id,
        SUM(pontos) AS num_pontos
    FROM todos_resultados
    GROUP BY time_id
)

SELECT 
    t.time_nome,
    COALESCE(p.num_pontos, 0) AS num_pontos
FROM datacraft_catalog.cantu.times t
LEFT JOIN pontuacao_por_time p ON t.time_id = p.time_id
ORDER BY num_pontos DESC, t.time_id ASC;

time_nome,num_pontos
Marketing,4
Dados,4
Financeiro,3
Logística,3
TI,0


### 1.2 Comissões

A tabela `comissoes` tem a seguinte estrutura:

```sql
CREATE TABLE comissoes (
    comprador VARCHAR NOT NULL,
    vendedor VARCHAR NOT NULL,
    dataPgto DATE NOT NULL,
    valor FLOAT NOT NULL
);
````

---

## 📌 Regras da Consulta

Escreva uma query SQL que retorne a **lista de vendedores** que:

* Receberam **pelo menos R\$ 1.024,00** em comissões;
* Considerando **no máximo 3 transferências** por vendedor;
* Ou seja, só deve aparecer na lista quem tiver até 3 registros e a soma total dos valores seja maior ou igual a R\$ 1.024,00;
* Vendedores com mais de 3 comissões **não devem aparecer**, mesmo que tenham recebido mais de R\$ 1.024,00;
* O resultado deve ser ordenado **em ordem alfabética** pelo nome do vendedor.

---

## 🧪 Exemplo de Dados

| Comprador | Vendedor | Data       | Valor    |
| --------- | -------- | ---------- | -------- |
| Leonardo  | Bruno    | 01/01/2000 | 200,00   |
| Leonardo  | Matheus  | 27/09/2003 | 1.024,00 |
| Leonardo  | Lucas    | 26/06/2006 | 512,00   |
| Marcos    | Lucas    | 17/12/2020 | 100,00   |
| Marcos    | Lucas    | 22/03/2002 | 10,00    |
| Cinthia   | Lucas    | 20/03/2021 | 500,00   |
| Mateus    | Bruno    | 02/06/2007 | 400,00   |
| Mateus    | Bruno    | 26/06/2006 | 400,00   |
| Mateus    | Bruno    | 26/06/2015 | 200,00   |

---

## ✅ Resultado Esperado

| Vendedor |
| -------- |
| Lucas    |
| Matheus  |

---

### ℹ️ Explicação

* **Lucas** tem 3 comissões: 512 + 100 + 500 = 1.112 ≥ 1.024 → OK ✅
* **Matheus** tem 1 comissão de 1.024 → OK ✅
* **Bruno** tem 4 comissões (mesmo somando 1.200, excede 3 transferências) → ❌

---


#### 1.2.1 Cria tabelas no Databricks - com Unity Catalog

In [0]:
%sql
USE datacraft_catalog.cantu;
CREATE TABLE IF NOT EXISTS cantu.comissoes (
  comissao_id LONG NOT NULL GENERATED ALWAYS AS IDENTITY(START WITH 1 INCREMENT BY 1),
  comprador STRING NOT NULL,
  vendedor STRING NOT NULL,
  dataPgto DATE NOT NULL,
  valor DOUBLE NOT NULL,
  PRIMARY KEY(comissao_id)
);

#### 1.2.2 Insere os dados na tabela no Databricks - com Unity Catalog

In [0]:
# Define o schema
schema = StructType([
    StructField("comprador", StringType(), False),
    StructField("vendedor", StringType(), False),
    StructField("dataPgto", DateType(), False),
    StructField("valor", FloatType(), False)
])

# Dados da tabela comissoes
dados_comissoes = [
    ("Leonardo", "Bruno",   datetime(2000, 1, 1),   200.00),
    ("Leonardo", "Matheus",datetime(2003, 9, 27), 1024.00),
    ("Leonardo", "Lucas",  datetime(2006, 6, 26),  512.00),
    ("Marcos",   "Lucas",  datetime(2020, 12, 17), 100.00),
    ("Marcos",   "Lucas",  datetime(2002, 3, 22),   10.00),
    ("Cinthia",  "Lucas",  datetime(2021, 3, 20),  500.00),
    ("Mateus",   "Bruno",  datetime(2007, 6, 2),   400.00),
    ("Mateus",   "Bruno",  datetime(2006, 6, 26),  400.00),
    ("Mateus",   "Bruno",  datetime(2015, 6, 26),  200.00),
]

# Cria o DataFrame
df_comissoes = spark.createDataFrame(dados_comissoes, schema)

# Cria view temporária
df_comissoes.createOrReplaceTempView('temp_comissoes')


In [0]:
%sql
INSERT INTO datacraft_catalog.cantu.comissoes (
    comprador, vendedor, dataPgto, valor
  )
SELECT
  comprador,
  vendedor,
  dataPgto,
  valor
FROM temp_comissoes

num_affected_rows,num_inserted_rows
9,9


In [0]:
%sql
WITH ranked_comissoes AS (
    SELECT
        vendedor,
        valor,
        ROW_NUMBER() OVER (PARTITION BY vendedor ORDER BY valor DESC) AS rn
    FROM datacraft_catalog.cantu.comissoes
),
top3_comissoes AS (
    SELECT
        vendedor,
        valor
    FROM ranked_comissoes
    WHERE rn <= 3
),
vendedores_qualificados AS (
    SELECT
        vendedor,
        COUNT(*) AS qtd_transferencias,
        SUM(valor) AS total_recebido
    FROM top3_comissoes
    GROUP BY vendedor
    HAVING qtd_transferencias <= 3 AND total_recebido >= 1024
)
SELECT vendedor
FROM vendedores_qualificados
ORDER BY vendedor ASC

vendedor
Lucas
Matheus


### 1.3 Organização Empresarial
---
## 📋 Estrutura da Tabela

```sql
CREATE TABLE colaboradores (
    id INTEGER NOT NULL,
    nome VARCHAR NOT NULL,
    salario INTEGER NOT NULL,
    lider_id INTEGER,
    UNIQUE(id)
);
````

> *A coluna `lider` foi ajustada para receber dados nulos, por conta dos requisitos da questão.*

---

## 🧠 Desafio

Para cada funcionário, retornar:

* `id` do funcionário;
* `id` do **chefe indireto** que:

  * Ganha pelo menos o **dobro do salário** do funcionário;
  * É um **chefe indireto** (ou seja, chefe do chefe, ou superior a isso);
* Se **nenhum chefe indireto** atender a condição, o valor deve ser `NULL`.

### 🧩 Definições

* Um funcionário A é **chefe indireto** de B se:

  * A é o chefe do chefe de B, ou do chefe do chefe do chefe... e assim por diante;
* Não existem ciclos na hierarquia (garantido pelo enunciado);
* O resultado deve ser ordenado por `id` do funcionário de forma crescente.

---

## 💡 Exemplo de Funcionários

- Diagrama Hierárquico:

```planText
Marcos (20) - $ 10.000
└── Leonardo (10) - $ 4.500
    ├── Bruno (50) - $ 3.000 
    │   ├── Helen  (40) - $ 1.500
    │   └── Wilian (30) - $ 1.501
    └── Mateus (70) - $ 1.500
        └── Cinthia (60) $ 2.000
```

| Id | Nome     | Salário | Lider\_Id |
| -- | -------- | ------- | --------- |
| 40 | Helen    | 1500    | 50        |
| 50 | Bruno    | 3000    | 10        |
| 10 | Leonardo | 4500    | 20        |
| 20 | Marcos   | 10000   | NULL      |
| 70 | Mateus   | 1500    | 10        |
| 60 | Cinthia  | 2000    | 70        |
| 30 | Wilian   | 1501    | 50        |

---



#### 1.3.1 Cria tabelas no Databricks - com Unity Catalog

In [0]:
%sql
USE datacraft_catalog.cantu;
CREATE TABLE IF NOT EXISTS cantu.colaboradores (
  id LONG NOT NULL, 
  nome STRING NOT NULL,
  salario DOUBLE NOT NULL,
  lider_id LONG,
  PRIMARY KEY(id)
);

In [0]:
# %sql
# DROP TABLE cantu.colaboradores

#### 1.1.2 Insere os dados nas tabelas no Databricks - com Unity Catalog

In [0]:
# Define o schema
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("nome", StringType(), False),
    StructField("salario", DoubleType(), False),
    StructField("lider_id", IntegerType(), True)
])

# Dados conforme a tabela da imagem
dados_colaboradores = [
    (40, "Helen",    1500, 50),
    (50, "Bruno",    3000, 10),
    (10, "Leonardo", 4500, 20),
    (20, "Marcos",  10000, None),
    (70, "Mateus",   1500, 10),
    (60, "Cinthia",  2000, 70),
    (30, "Wilian",   1501, 50),
]

# Cria o DataFrame
df_colaboradores = spark.createDataFrame(dados_colaboradores, schema)

# Exibe o conteúdo
df_colaboradores.createOrReplaceTempView('temp_colaboradores')


In [0]:
%sql
INSERT INTO datacraft_catalog.cantu.colaboradores (
    id, nome, salario, lider_id
  )
SELECT
  id,
  nome,
  salario,
  lider_id
FROM temp_colaboradores

num_affected_rows,num_inserted_rows
7,7


In [0]:
%sql
WITH chefe_direto AS (
  -- Retorna o chefe direto de cada funcionário
  SELECT
    id,
    nome,
    salario,
    lider_id as id_chefe_direto
  FROM datacraft_catalog.cantu.colaboradores
), chefe_indireto AS (
  -- Retorna o chefe indireto de cada funcionário
  SELECT
    chefe_direto.*,
    chefe.nome as nome_chefe_direto,
    chefe.salario as salario_chefe_direto,
    chefe.lider_id as id_chefe_indireto
  FROM chefe_direto
  LEFT JOIN datacraft_catalog.cantu.colaboradores AS chefe ON chefe.id = chefe_direto.id_chefe_direto
)
-- retorna o id_funcionario e o id_do_chefe_indireto
SELECT
  id as id_funcionario,
  id_chefe_indireto
FROM chefe_indireto
ORDER BY 1

id_funcionario,id_chefe_indireto
10,null
20,null
30,10
40,10
50,20
60,10
70,20
